In [1]:
import xarray as xr
import numpy as np
import os
from tqdm import tqdm

In [2]:
# Input and output directories
input_dir = r'C:\Users\Admin\Desktop\Anurag-6048 Part-3\EOS-Source-Input-Data\EOS-MOSAIC_INPUT-MAY-2024'
output_dir = r'C:\Users\Admin\Desktop\Anurag-6048 Part-3\EOS-CSV_INPUT-MAY\MAY-RRS'

In [3]:
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Loop through all files in the input directory
for root, dirs, files in os.walk(input_dir):
    for filename in tqdm(files, desc="Processing Files"):
        if filename.endswith('.nc'):
            file_path = os.path.join(root, filename)
            print(f"Processing: {filename}")

            # Open the NetCDF file using xarray
            ds = xr.open_dataset(file_path)

            # Extract variables
            rrs01 = ds['RRS01']
            rrs05 = ds['RRS05']
            rrs02 = ds['RRS03']
            rrs03 = ds['RRS02']
            rrs07 = ds['RRS07']

            # Perform calculations (example: calculate RRS01 / RRS05)
            output_x = xr.where((rrs01 > 0) & (rrs05 > 0),
                                0.1082 * (rrs01 / rrs05) ** -0.968,
                                np.nan)
            
            x1=xr.where((rrs02>0)&(rrs03>0)&(rrs05>0),((rrs03/rrs05)-rrs02/(rrs03/rrs05)+rrs02),np.nan)

            x2=np.power(10,(-0.1069*x1*x1*x1+0.6259*x1*x1-1.3936*x1+0.919))

            x2s= xr.where(rrs02>0,
                      np.power(10, (-0.1069 * rrs02 * rrs02 * rrs02 + 0.6259 * rrs02 * rrs02 - 1.3936 * rrs02 + 0.919)),
                      np.nan)

            rt=np.power(10,x2)

            docr5r7 = xr.where((rrs05 > 0) & (rrs07 > 0),89.86*np.exp((0.27)*(rrs07/ rrs05)),np.nan)

            # Add the new variable to the dataset
            ds['CDOM'] = output_x
            ds['X1'] = x1
            ds['X2'] = x2
            ds['RT'] = rt
            ds['X2-S'] = x2s
            ds['DOC'] = docr5r7

            # Write the updated dataset to a new NetCDF file
            output_file = os.path.join(output_dir, f"{filename.split('.nc')[0]}_cdom_rt_processed.nc")
            ds.to_netcdf(output_file)
            print(f"Output file generated: {output_file}")

            # Close the dataset
            ds.close()